In [1]:
import os
import pandas as pd
import numpy as np

In [ ]:
final_path = '/content/drive/MyDrive/DMS/data/splits/24frame_windows'
final_train_df = pd.read_csv(os.path.join(final_path, 'final_train.csv'))
final_val_df = pd.read_csv(os.path.join(final_path, 'final_val.csv'))
final_test_df = pd.read_csv(os.path.join(final_path, 'final_test.csv'))

<ipython-input-2-b774389ab440>:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  final_train_df = pd.read_csv(os.path.join(final_path, 'final_train.csv'))


In [2]:
mar_df = pd.read_csv('/content/drive/MyDrive/DMS/data/features/mar_ear_final/yawn_text_interpolated.csv')
gaze_df = pd.read_csv('/content/drive/MyDrive/DMS/data/features/yaw_pitch_final/yawn_text_interpolated.csv')

In [ ]:
final_train_df.shape, final_val_df.shape, final_test_df.shape

((81648, 14), (14784, 14), (23736, 14))

In [ ]:
mar_df.shape, gaze_df.shape

((21303, 13), (21303, 13))

In [ ]:
mar_df.columns, gaze_df.columns

(Index(['frame_path', 'label', 'camera', 'clip_id', 'frame_no', 'MAR',
        'MAR_horiz_dist', 'MAR_vert_mean', 'mouth_landmarks', 'EAR',
        'EAR_horiz_dist', 'EAR_vert_mean', 'eyes_landmarks'],
       dtype='object'),
 Index(['frame_path', 'label', 'camera', 'clip_id', 'frame_no',
        'local_frame_path', 'yaw', 'pitch', 'bbox', 'landmarks', 'dataset',
        'MAR', 'EAR'],
       dtype='object'))

In [3]:
mar_gaze_df = mar_df.merge(gaze_df[['frame_path', 'yaw', 'pitch', 'dataset']], on='frame_path', how='left')

In [ ]:
mar_gaze_df.shape

(21303, 16)

In [ ]:
mar_gaze_df.columns

Index(['frame_path', 'label', 'camera', 'clip_id', 'frame_no', 'MAR',
       'MAR_horiz_dist', 'MAR_vert_mean', 'mouth_landmarks', 'EAR',
       'EAR_horiz_dist', 'EAR_vert_mean', 'eyes_landmarks', 'yaw', 'pitch',
       'dataset'],
      dtype='object')

# train

In [ ]:
final_train_df.head()

,frame_path,label,camera,clip_id,frame_no,dataset,groups_24,mar,ear,yaw,pitch,phone_detected,seven_group,subject_id
0,/content/drive/MyDrive/DMS/data/processed/dmd_...,normal,NaN,gE-27(File17)_s1_face_2019-03-07-13;14;28_27_0...,0,DMD,0,0.097440,0.123009,-0.747302,0.109527,0,DMD_normal,NaN
1,/content/drive/MyDrive/DMS/data/processed/dmd_...,normal,NaN,gE-27(File17)_s1_face_2019-03-07-13;14;28_27_0...,1,DMD,0,0.158535,0.142181,-1.102433,0.204548,0,DMD_normal,NaN
2,/content/drive/MyDrive/DMS/data/processed/dmd_...,normal,NaN,gE-27(File17)_s1_face_2019-03-07-13;14;28_27_0...,2,DMD,0,0.059623,0.145776,-0.834969,-0.191205,0,DMD_normal,NaN
3,/content/drive/MyDrive/DMS/data/processed/dmd_...,normal,NaN,gE-27(File17)_s1_face_2019-03-07-13;14;28_27_0...,3,DMD,0,0.143732,0.175720,-1.127011,0.237445,0,DMD_normal,NaN
4,/content/drive/MyDrive/DMS/data/processed/dmd_...,normal,NaN,gE-27(File17)_s1_face_2019-03-07-13;14;28_27_0...,4,DMD,0,0.245684,0.189049,-2.031260,-0.091060,0,DMD_normal,NaN


In [4]:
mar_gaze_df['phone_detected'] = 0

In [5]:
mar_gaze_df['seven_group'] = mar_gaze_df['dataset'] + '_' + mar_gaze_df['label']

In [6]:
mar_gaze_df['subject_id'] = np.nan

In [7]:
mar_gaze_df['groups_24'] = np.nan

In [8]:
mar_gaze_df.columns = [col.lower() for col in mar_gaze_df.columns]

In [9]:
mar_gaze_df = mar_gaze_df[['frame_path', 'label', 'camera', 'clip_id', 'frame_no', 'dataset',
       'groups_24', 'mar', 'ear', 'yaw', 'pitch', 'phone_detected',
       'seven_group', 'subject_id']]

In [ ]:
# 1. 공통된 (label, clip_id)만 필터링
common_keys = set(map(tuple, final_train_df[['label', 'clip_id']].drop_duplicates().values))
mar_gaze_common = mar_gaze_df[mar_gaze_df[['label', 'clip_id']].apply(tuple, axis=1).isin(common_keys)]

# 2. (label, clip_id, frame_no) 기준으로 차집합
set1 = set(map(tuple, mar_gaze_common[['label', 'clip_id', 'frame_no']].values))
set2 = set(map(tuple, final_train_df[['label', 'clip_id', 'frame_no']].values))
diff = set1 - set2

# 3. 차집합을 DataFrame으로 변환
diff_df = pd.DataFrame(list(diff), columns=['label', 'clip_id', 'frame_no'])

# 4. 해당 행 추출
diff_rows = mar_gaze_common.merge(diff_df, on=['label', 'clip_id', 'frame_no'], how='inner')

# 5. concat
final_train_concat_df = pd.concat([final_train_df, diff_rows], ignore_index=True)

In [ ]:
final_train_df.shape

(81648, 14)

In [ ]:
final_train_concat_df.shape

(83451, 14)

In [ ]:
len(final_train_df[['label', 'clip_id']].drop_duplicates()),\
len(final_train_concat_df[['label', 'clip_id']].drop_duplicates())

(1346, 1346)

In [ ]:
origin_path = os.path.join(final_path, 'final_train.csv')
new_path = os.path.join(final_path, 'final_train_v2.csv')

os.rename(origin_path, new_path)

In [ ]:
final_train_concat_df.to_csv(origin_path, index=False)

# val

In [ ]:
# 1. 공통된 (label, clip_id)만 필터링
common_keys = set(map(tuple, final_val_df[['label', 'clip_id']].drop_duplicates().values))
mar_gaze_common = mar_gaze_df[mar_gaze_df[['label', 'clip_id']].apply(tuple, axis=1).isin(common_keys)]

# 2. (label, clip_id, frame_no) 기준으로 차집합
set1 = set(map(tuple, mar_gaze_common[['label', 'clip_id', 'frame_no']].values))
set2 = set(map(tuple, final_val_df[['label', 'clip_id', 'frame_no']].values))
diff = set1 - set2

# 3. 차집합을 DataFrame으로 변환
diff_df = pd.DataFrame(list(diff), columns=['label', 'clip_id', 'frame_no'])

# 4. 해당 행 추출
diff_rows = mar_gaze_common.merge(diff_df, on=['label', 'clip_id', 'frame_no'], how='inner')

# 5. concat
final_val_concat_df = pd.concat([final_val_df, diff_rows], ignore_index=True)

In [ ]:
final_val_df.shape

(14784, 14)

In [ ]:
final_val_concat_df.shape

(15085, 14)

In [ ]:
len(final_val_df[['label', 'clip_id']].drop_duplicates()),\
len(final_val_concat_df[['label', 'clip_id']].drop_duplicates())

(240, 240)

In [ ]:
origin_path = os.path.join(final_path, 'final_val.csv')
new_path = os.path.join(final_path, 'final_val_v2.csv')

os.rename(origin_path, new_path)
final_val_concat_df.to_csv(origin_path, index=False)

# test

In [10]:
final_path = '/content/drive/MyDrive/DMS/data/splits/24frame_windows'
final_test_df = pd.read_csv(os.path.join(final_path, 'final_test_v2.csv'))

In [11]:
# 1. 공통된 (label, clip_id)만 필터링
common_keys = set(map(tuple, final_test_df[['label', 'clip_id']].drop_duplicates().values))
mar_gaze_common = mar_gaze_df[mar_gaze_df[['label', 'clip_id']].apply(tuple, axis=1).isin(common_keys)]

# 2. (label, clip_id, frame_no) 기준으로 차집합
set1 = set(map(tuple, mar_gaze_common[['label', 'clip_id', 'frame_no']].values))
set2 = set(map(tuple, final_test_df[['label', 'clip_id', 'frame_no']].values))
diff = set1 - set2

# 3. 차집합을 DataFrame으로 변환
diff_df = pd.DataFrame(list(diff), columns=['label', 'clip_id', 'frame_no'])

# 4. 해당 행 추출
diff_rows = mar_gaze_common.merge(diff_df, on=['label', 'clip_id', 'frame_no'], how='inner')

# 5. concat
final_test_concat_df = pd.concat([final_test_df, diff_rows], ignore_index=True)

In [ ]:
final_test_df.shape

(23736, 14)

In [ ]:
final_test_concat_df.shape

(24226, 14)

In [15]:
len(final_test_df[['label', 'clip_id']].drop_duplicates()),\
len(final_test_concat_df[['label', 'clip_id']].drop_duplicates())

(425, 425)

In [ ]:
origin_path = os.path.join(final_path, 'final_test.csv')
new_path = os.path.join(final_path, 'final_test_v2.csv')

os.rename(origin_path, new_path)
final_test_concat_df.to_csv(origin_path, index=False)

In [16]:
origin_path = os.path.join(final_path, 'final_test.csv')
final_test_concat_df.to_csv(origin_path, index=False)

In [17]:
check_test = pd.read_csv(origin_path)

In [18]:
check_test.shape

(24226, 14)

In [ ]:
final_path

'/content/drive/MyDrive/DMS/data/splits/24frame_windows'